<a href="https://colab.research.google.com/github/andssuu/moncattle/blob/master/supervised/notebooks/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/andssuu/moncattle.git

Cloning into 'moncattle'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 128 (delta 42), reused 82 (delta 28), pack-reused 0
Receiving objects: 100% (128/128), 980.58 KiB | 15.32 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [6]:
import random

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


def sigmoid(net):
    return 1.0/(1.0+np.exp(-net))


def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))


class Network(object):

    def __init__(self, sizes, activation_function=sigmoid,
                 prime_function=sigmoid_prime):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(
            sizes[:-1], sizes[1:])]
        self.activation_function = activation_function
        self.prime_function = prime_function

    def feedforward(self, x):
        """Retorna a saída da rede z se `x` for entrada."""
        for b, w in zip(self.biases, self.weights):
            x = self.activation_function(np.dot(w, x)+b)  # net = (∑xw+b)
        return x

    def SGD(self, training_data, epochs, mini_batch_size, _n, test_data=None):
        n = len(training_data)
        n_test = len(test_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size]
                            for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, _n)
            if test_data:
                acc = self.evaluate(test_data)
                print("Epoch {} : {} / {} = {}%".format(j,
                                                        acc, n_test, (acc*100)/n_test))
            else:
                print("Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, _n):
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        self.weights = [w-(_n/len(mini_batch))*nw for w,
                        nw in zip(self.weights, nabla_w)]
        self.biases = [b-(_n/len(mini_batch))*nb for b,
                       nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo J_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        activation = x
        activations = [x]
        nets = []
        for b, w in zip(self.biases, self.weights):
            net = np.dot(w, activation)+b
            nets.append(net)
            activation = self.activation_function(net)
            activations.append(activation)
        # Backward pass
        delta = self.cost_derivative(
            activations[-1], y) * self.prime_function(nets[-1])
        nabla_b[-1] = delta
        # (𝑦−𝑧)*f’(net)*𝑥
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            net = nets[-l]
            zs = self.prime_function(net)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * zs
            nabla_b[-l] = delta
            # ∑(𝛿𝑤)f’(net)𝑥
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y): return (output_activations-y)


def load_data():
    url = 'moncattle/data/lomba.csv'
    df = pd.read_csv(url)
    data = df[df.columns[1:10]]
    # normaliza os dados
    normalized_data = (data - data.min()) / (data.max() - data.min())
    labels = df[df.columns[-1]]
    x_train, x_test, y_train, y_test = train_test_split(
        normalized_data, labels, test_size=0.4, random_state=0)
    le = preprocessing.LabelEncoder()
    le.fit(y_train.values)
    y_train = [vectorized_result(y) for y in le.transform(y_train.values)]
    x_train = [np.reshape(x, (9, 1)) for x in x_train.values]
    x_test = [np.reshape(x, (9, 1)) for x in x_test.values]
    return zip(x_train, y_train), zip(x_test, le.transform(y_test.values))


def vectorized_result(j):
    e = np.zeros((4, 1))
    e[j] = 1.0
    return e


if __name__ == "__main__":
    training_data, test_data = load_data()
    arquitecture = [9, 30, 20, 4]
    mlp = Network(arquitecture)
    mlp.SGD(list(training_data), 20, 24, 0.3, test_data=list(test_data))


Epoch 0 : 2316 / 5236 = 44.23223834988541%
Epoch 1 : 2927 / 5236 = 55.901451489686785%
Epoch 2 : 2994 / 5236 = 57.18105423987777%
Epoch 3 : 3331 / 5236 = 63.617265087853326%
Epoch 4 : 3670 / 5236 = 70.0916730328495%
Epoch 5 : 3541 / 5236 = 67.62796027501909%
Epoch 6 : 3792 / 5236 = 72.42169595110772%
Epoch 7 : 3801 / 5236 = 72.59358288770053%
Epoch 8 : 3773 / 5236 = 72.05882352941177%
Epoch 9 : 3576 / 5236 = 68.29640947288006%
Epoch 10 : 3684 / 5236 = 70.35905271199388%
Epoch 11 : 3741 / 5236 = 71.44766997708174%
Epoch 12 : 3802 / 5236 = 72.61268143621085%
Epoch 13 : 3741 / 5236 = 71.44766997708174%
Epoch 14 : 3704 / 5236 = 70.74102368220015%
Epoch 15 : 3800 / 5236 = 72.57448433919022%
Epoch 16 : 3732 / 5236 = 71.27578304048892%
Epoch 17 : 3750 / 5236 = 71.61955691367456%
Epoch 18 : 3678 / 5236 = 70.24446142093201%
Epoch 19 : 3742 / 5236 = 71.46676852559206%
